In [1]:
import numpy as np 
import pandas as pd 
import gc
import os
from sklearn.metrics import classification_report
from scipy import stats,sparse
import time
import os
import itertools
import math
from tqdm import *
from scipy.stats import entropy, pearsonr
import xgboost as xgb
import lightgbm as lgb
import catboost as cbt
print('lightgbm version :', lgb.__version__)
from sklearn.model_selection import StratifiedKFold,KFold,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score,accuracy_score,roc_auc_score
from sklearn.metrics import roc_auc_score,recall_score,accuracy_score,log_loss,precision_score
import multiprocessing
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.preprocessing import label_binarize, LabelBinarizer
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.stats import pearsonr,spearmanr,kendalltau,entropy
import warnings
warnings.filterwarnings('ignore')

lightgbm version : 2.2.3


#### 获取数据~~~~

In [3]:
def get_data():
    train_tag = pd.read_csv('../data/训练数据集_tag.csv')
    train_trd = pd.read_csv('../data/训练数据集_trd.csv')
    train_beh = pd.read_csv('../data/训练数据集_beh.csv')
    del train_beh['page_tm']
    train_beh.columns = ['id', 'flag', 'page_no', 'page_tm']
    print('train table: tag id {}, trd id {}, beh id {}'.format(train_tag.id.nunique(), train_trd.id.nunique(), train_beh.id.nunique()))
    print('all train id number: ', len(set(train_tag.id)|(set(train_trd.id)|set(train_beh.id))))
    # print(len(set(train_tag.id)&(set(train_trd.id)|set(train_beh.id))))
    
    print(train_tag.shape, train_trd.shape, train_beh.shape)

    test_tag = pd.read_csv('../data/评分数据集_tag_b.csv')
    test_trd = pd.read_csv('../data/评分数据集_trd_b.csv')
    test_beh = pd.read_csv('../data/评分数据集_beh_b.csv')
    del test_beh['page_tm']
    test_beh.columns = ['id', 'page_no', 'page_tm']
    print('test table: tag id {}, trd id {}, beh id {}'.format(test_tag.id.nunique(), test_trd.id.nunique(), test_beh.id.nunique()))
    # print(len(set(test_tag.id)&(set(test_trd.id)|set(test_beh.id))))
    print('all test id number: ', len(set(test_tag.id)|(set(test_trd.id)|set(test_beh.id))))
    print(test_tag.shape, test_trd.shape, test_beh.shape)
    
    tag = train_tag.append(test_tag, sort=False).reset_index(drop=True)
    trd = train_trd.append(test_trd, sort=False).reset_index(drop=True)
    beh = train_beh.append(test_beh, sort=False).reset_index(drop=True)
    return tag, trd, beh

In [4]:
tag, trd, beh = get_data()

train table: tag id 39923, trd id 31993, beh id 11913
all train id number:  39923
(39923, 43) (1367211, 8) (934282, 4)
test table: tag id 4000, trd id 3190, beh id 1232
all test id number:  4000
(4000, 42) (142645, 7) (95669, 3)


In [5]:
for i in ['trd_id', 'beh_id']:
    tag[i] = 0
    if i == 'trd_id':
        tag.loc[tag.id.isin(trd.id.unique()), i] = 1
    else:
        tag.loc[beh.id.isin(beh.id.unique()), i] = 1
tag['sample'] = tag[['trd_id', 'beh_id','flag']].astype(str).sum(axis=1)

#### **`tag`**

In [6]:
tag_num_cols = [i for i in tag.dtypes[tag.dtypes != object].index if i not in ['id','flag','trd_id', 'beh_id','sample']]
tag_cate_cols = [i for i in tag.columns if i not in tag_num_cols+['id','flag', 'trd_id', 'beh_id','sample']]
# ind_col = [i for i in tag.columns if 'ind' in i]
# cd_col = [i for i in tag.columns if 'cd' in i]
# for i in tag.columns:
#     for j in ['\\N', '-1', -1, '~']:
#         tag[i] = tag[i].replace(j, np.nan)

In [7]:
cate_to_num_col = []
for col in tag_cate_cols:
    try:
        tag[col] = tag[col].replace('\\N', np.nan)
        tag[col] = tag[col].astype('float')
        # print(col)
        cate_to_num_col.append(col)
    except:
        pass

In [8]:
#类别更改为labelencoder
tag_cate_cols = [i for i in tag_cate_cols if i not in cate_to_num_col+['gdr_cd']]
tag['gdr_cd'] = tag['gdr_cd'].map({'M':1,'F':0})
# tag = pd.get_dummies(columns=tag_cate_cols,data=tag)
# tag = tag.replace(-1,np.nan)
# tag = tag.drop(columns=tag_cate_cols)

In [9]:
"""理财"""
# l12_col = ['l12mon_buy_fin_mng_whl_tms', 'l12_mon_fnd_buy_whl_tms', 'l12_mon_gld_buy_whl_tms']
# tag['l12mon_buy_sum'] = tag[l12_col].sum(axis=1)
# for col in l12_col:
#     tag[f'{col}_ratio'] = tag[col] / (tag['l12mon_buy_sum']+1)

'理财'

In [10]:
cur_col = ['cur_debit_cnt', 'cur_credit_cnt', 'cur_debit_min_opn_dt_cnt',
'cur_credit_min_opn_dt_cnt', 'cur_debit_crd_lvl','hld_crd_card_grd_cd']
tag.loc[tag['cur_credit_min_opn_dt_cnt']==-1,'hld_crd_card_grd_cd'] = -1
for col in [ 'cur_debit_min_opn_dt_cnt', 'cur_credit_min_opn_dt_cnt',]:
    tag[col] = tag[col].replace(-1,0)
tag['debit-credit'] = tag['cur_debit_min_opn_dt_cnt'] - tag['cur_credit_min_opn_dt_cnt']
# tag['credit/debit'] = tag['cur_debit_cnt'] / (tag['cur_credit_cnt']+1)

#### **trd**

In [11]:
def pivot_fe(data, value, ind, column, func, kk):
    pivot = pd.pivot_table(data, values=[value], index=ind, columns=column,aggfunc=func)
    pivot.columns = [value + f'_{i}_{j}_{kk}' for i in pivot.columns.levels[0] for j in pivot.columns.levels[2]]
    pivot = pivot.reset_index().fillna(0)
    return pivot

In [12]:
for col in ['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd']:
    temp = pivot_fe(trd, 'cny_trx_amt', 'id', col, ['max', 'min', 'std', 'mean', 'median', 'sum','count'], col)
    print(temp.shape)
    temp.columns = ['id'] + temp.columns.tolist()[1:]
    print(temp.shape)
    tag = tag.merge(temp, on = 'id', how='left')

(35183, 15)
(35183, 15)
(35183, 22)
(35183, 22)
(35183, 22)
(35183, 22)


In [13]:
# for i in ['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd']:
#     trd[f'{i}_count'] = trd.groupby(i)['id'].transform('count')
trd['Trx_Cod2_Cd_count'] = trd['Trx_Cod2_Cd'].map(trd['Trx_Cod2_Cd'].value_counts())

In [14]:
aggs = {}
for i in ['Dat_Flg1_Cd', 'Dat_Flg3_Cd', 'Trx_Cod1_Cd', 'Trx_Cod2_Cd']:
    # aggs[f'{i}_count'] = ['min']
    if i == 'Dat_Flg1_Cd':
        aggs[i] = ['nunique', 'count']
    else:
        aggs[i] = ['nunique']
aggs['Trx_Cod2_Cd_count'] = ['max', 'min', 'std', 'mean']
aggs['cny_trx_amt'] = ['sum', 'mean']
agg_df=trd.groupby('id').agg(aggs)
agg_df.columns = ['trd'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()
tag = tag.merge(agg_df, on = 'id', how='left')

In [15]:
for i in ['cny_trx_amt_count_B_Dat_Flg1_Cd', 'cny_trx_amt_count_C_Dat_Flg1_Cd',
 'cny_trx_amt_count_A_Dat_Flg3_Cd', 'cny_trx_amt_count_B_Dat_Flg3_Cd',
 'cny_trx_amt_count_C_Dat_Flg3_Cd', 'cny_trx_amt_count_1_Trx_Cod1_Cd',
 'cny_trx_amt_count_2_Trx_Cod1_Cd', 'cny_trx_amt_count_3_Trx_Cod1_Cd']:
    tag[f'{i}_ratio'] = tag[i] / tag['trd_Dat_Flg1_Cd_count']

In [16]:
"""tfidf"""
feat = trd[['id','Trx_Cod2_Cd']]
feat['Trx_Cod2_Cd'] = feat['Trx_Cod2_Cd'].astype(str)
feat = feat.groupby(['id'])['Trx_Cod2_Cd'].agg(lambda x: ' '.join(x)).reset_index(name='Trx_Cod2_Cd_list')
tf_idf = TfidfVectorizer(max_features=30)
tf_vec = tf_idf.fit_transform(feat['Trx_Cod2_Cd_list'].values.tolist())
# svd = TruncatedSVD(n_components=16, n_iter=10, random_state=2019)
# svd_vec = svd.fit_transform(tf_vec)
tf_df = pd.DataFrame(tf_vec.toarray())
tf_df['id'] = feat['id'].values
tf_df.columns = ['Trx_Cod2_Cd_svd_vec_'+str(i+1) for i in range(30)]+ ['id']
tag = tag.merge(tf_df,on='id', how='left')
del tf_df,feat;gc.collect()

22

In [17]:
"""时间统计"""
trd['trx_tm'] = pd.to_datetime(trd['trx_tm'])
trd['trx_tm_mon'] = trd['trx_tm'].dt.month
trd['trx_tm_day'] = trd['trx_tm'].dt.day
trd['trx_tm_dayofyear'] = trd['trx_tm'].dt.dayofyear
trd['trx_tm_dayofyear'] = trd['trx_tm_dayofyear'] - trd['trx_tm_dayofyear'].min()
trd['trx_tm_hour'] = trd['trx_tm'].dt.hour
trd['trx_tm_weekday'] = trd['trx_tm'].dt.weekday
trd['trx_tm_wy'] = trd['trx_tm'].dt.weekofyear
trd['trx_tm_wy'] = trd['trx_tm_wy'] - trd['trx_tm_wy'].min()
trd['trx_tm_是否周末'] = trd['trx_tm_weekday'].apply(lambda x:1 if x >= 5 else 0)

In [18]:
aggs = {
    'trx_tm_dayofyear':['nunique', 'max', 'min', np.ptp],
}
agg_df=trd.groupby('id').agg(aggs)
agg_df.columns = ['trd'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()  
tag = tag.merge(agg_df, on = 'id', how='left') 

In [19]:
#按月统计 sum
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_mon', ['sum'], 'month')
temp['5-6'] = temp['cny_trx_amt_sum_5_month'] - temp['cny_trx_amt_sum_6_month']
tag = tag.merge(temp,on='id', how='left')
#按工作日统计 mean
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_是否周末', ['mean'], '是否周末')
tag = tag.merge(temp,on='id', how='left')
# #按1-7天统计
# temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_weekday', ['mean'], 'weekday')
# tag = tag.merge(temp,on='id', how='left')
# #按小时天统计
# temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_hour', ['mean'], 'hour')
# tag = tag.merge(temp, on = 'id', how='left')

In [20]:
# 按周统计 'sum','mean'
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_wy', ['sum','mean'], 'wy')
tag = tag.merge(temp,on='id', how='left')

temp = trd.groupby(['id', 'trx_tm_wy'])['cny_trx_amt'].sum().reset_index(name = 'sum')
tag['trx_tm_wy_spearmanr_wy'] = tag['id'].map(temp.groupby('id').apply(lambda x:spearmanr(x['trx_tm_wy'], x['sum'])[0]))

temp = trd.groupby(['id', 'trx_tm_wy'])['cny_trx_amt'].count().reset_index(name = 'count')
tag['trx_tm_wy_spearmanr_count'] = tag['id'].map(temp.groupby('id').apply(lambda x:spearmanr(x['trx_tm_wy'], x['count'])[0]))

In [21]:
temp = pivot_fe(trd, 'cny_trx_amt', 'id', 'trx_tm_wy', ['count'], 'wy')
temp.drop(columns=['cny_trx_amt_count_2_wy',
       'cny_trx_amt_count_3_wy', 'cny_trx_amt_count_4_wy',
       'cny_trx_amt_count_5_wy', 'cny_trx_amt_count_6_wy',
       'cny_trx_amt_count_7_wy'], inplace=True)
tag = tag.merge(temp,on='id', how='left')
for i in temp.columns[1:]:
    tag[i] = tag[i] / tag['trd_Dat_Flg1_Cd_count']

In [22]:
temp = pd.pivot_table(data=trd,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='nunique').reset_index()
temp.columns = ['id'] + [f'trx_tm_mon_{i}' for i in [5,6]]
temp['trx_tm_mon_mean'] = (temp['trx_tm_mon_5'] + temp['trx_tm_mon_6'])/2
temp['trx_tm_mon_ratio'] = (temp['trx_tm_mon_6'] / temp['trx_tm_mon_5'])
temp.columns = ['id'] + ['trd_nunique_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='count').reset_index()
temp.columns = ['id'] + [f'ctrx_tm_mon_{i}' for i in [5,6]]
temp['ctrx_tm_mon_mean'] = (temp['ctrx_tm_mon_5'] + temp['ctrx_tm_mon_6'])/2
temp['ctrx_tm_mon_ratio'] = (temp['ctrx_tm_mon_6'] / temp['ctrx_tm_mon_5'])
temp.columns = ['id'] + ['trd_count_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd,index='id', columns='trx_tm_mon', values='Trx_Cod2_Cd',aggfunc='nunique').reset_index()
temp.columns = ['id'] + [f'trx_tm_mon_Trx_Cod2_Cd_{i}' for i in [5,6]]
temp['trx_tm_mon_Trx_Cod2_Cd_mean'] = (temp['trx_tm_mon_Trx_Cod2_Cd_5'] + temp['trx_tm_mon_Trx_Cod2_Cd_6'])/2
temp['trx_tm_mon_Trx_Cod2_Cd_ratio'] = (temp['trx_tm_mon_Trx_Cod2_Cd_6'] / temp['trx_tm_mon_Trx_Cod2_Cd_5'])
temp.columns = ['id'] + ['trd_Trx_Cod2_Cd_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

In [23]:
trd_b = trd[trd['Dat_Flg1_Cd'] == 'B'].copy()
trd_c = trd[trd['Dat_Flg1_Cd'] == 'C'].copy()

In [24]:
temp = pd.pivot_table(data=trd_b,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='nunique').reset_index()
temp.columns = ['id'] + [f'btrx_tm_mon_{i}' for i in [5,6]]
temp['btrx_tm_mon_mean'] = (temp['btrx_tm_mon_5'] + temp['btrx_tm_mon_6'])/2
temp['btrx_tm_mon_ratio'] = (temp['btrx_tm_mon_6'] / temp['btrx_tm_mon_5'])
temp.columns = ['id'] + ['trd_b_nunique_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd_b,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='count').reset_index()
temp.columns = ['id'] + [f'bctrx_tm_mon_{i}' for i in [5,6]]
temp['bctrx_tm_mon_mean'] = (temp['bctrx_tm_mon_5'] + temp['bctrx_tm_mon_6'])/2
temp['bctrx_tm_mon_ratio'] = (temp['bctrx_tm_mon_6'] / temp['bctrx_tm_mon_5'])
temp.columns = ['id'] + ['trd_b_count_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd_b,index='id', columns='trx_tm_mon', values='cny_trx_amt',aggfunc='sum').reset_index()
temp.columns = ['id'] + [f'trx_tm_mon_{i}' for i in [5,6]]
temp['trx_tm_mon_mean'] = (temp['trx_tm_mon_5'] + temp['trx_tm_mon_6'])/2
temp['trx_tm_mon_ratio'] = (temp['trx_tm_mon_6'] / temp['trx_tm_mon_5'])
temp.columns = ['id'] + ['trd_b_amt_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

In [25]:
temp = pd.pivot_table(data=trd_c,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='nunique').reset_index()
temp.columns = ['id'] + [f'cntrx_tm_mon_{i}' for i in [5,6]]
temp['cntrx_tm_mon_mean'] = (temp['cntrx_tm_mon_5'] + temp['cntrx_tm_mon_6'])/2
temp['cntrx_tm_mon_ratio'] = (temp['cntrx_tm_mon_6'] / temp['cntrx_tm_mon_5'])
temp.columns = ['id'] + ['trd_c_nunique_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd_c,index='id', columns='trx_tm_mon', values='trx_tm_dayofyear',aggfunc='count').reset_index()
temp.columns = ['id'] + [f'bctrx_tm_mon_{i}' for i in [5,6]]
temp['bctrx_tm_mon_mean'] = (temp['bctrx_tm_mon_5'] + temp['bctrx_tm_mon_6'])/2
temp['bctrx_tm_mon_ratio'] = (temp['bctrx_tm_mon_6'] / temp['bctrx_tm_mon_5'])
temp.columns = ['id'] + ['trd_c_count_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = pd.pivot_table(data=trd_c,index='id', columns='trx_tm_mon', values='cny_trx_amt',aggfunc='sum').reset_index()
temp.columns = ['id'] + [f'trx_tm_mon_{i}' for i in [5,6]]
temp['trx_tm_mon_mean'] = (temp['trx_tm_mon_5'] + temp['trx_tm_mon_6'])/2
temp['trx_tm_mon_ratio'] = (temp['trx_tm_mon_6'] / temp['trx_tm_mon_5'])
temp.columns = ['id'] + ['trd_c_amt_'+i for i in temp.columns[1:]]
tag = tag.merge(temp, on = 'id', how = 'left')

In [26]:
tag['xiaofeixingwei'] = tag['trd_b_amt_trx_tm_mon_mean'] / tag['trd_c_amt_trx_tm_mon_mean']

In [27]:
def get_cons_day(x):
    cc = np.sort(np.unique(x))
    if len(cc)==1:
        return 1
    bb = list(np.diff(cc))
    dd = []
    kz = []
    for k, v in itertools.groupby(bb):
        kz.append(k)
        if not kz.count(1):
            return 1
        elif k==1:
            dd.append(len(list(v)))
    return [i+1 for i in dd]#np.max(dd)+1

In [28]:
temp = trd_b.groupby('id')['trx_tm_dayofyear'].apply(get_cons_day).reset_index(name='cons')
temp['cons_b_max'] = temp['cons'].apply(lambda x:np.max(x))
temp['cons_b_std'] = temp['cons'].apply(lambda x:np.std(x))
temp['cons_b_mean'] = temp['cons'].apply(lambda x:np.mean(x))
del temp['cons']
tag = tag.merge(temp, on = 'id', how='left')

temp = trd_c.groupby('id')['trx_tm_dayofyear'].apply(get_cons_day).reset_index(name='cons')
temp['cons_c_max'] = temp['cons'].apply(lambda x:np.max(x))
temp['cons_c_std'] = temp['cons'].apply(lambda x:np.std(x))
temp['cons_c_mean'] = temp['cons'].apply(lambda x:np.mean(x))
del temp['cons']
tag = tag.merge(temp, on = 'id', how='left')

In [29]:
aggs = {
    'trx_tm_dayofyear':['nunique', 'max', 'min', np.ptp],
}
agg_df=trd_b.groupby('id').agg(aggs)
agg_df.columns = ['trd_b'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()  
tag = tag.merge(agg_df, on = 'id', how='left') 

In [30]:
temp = trd_b.groupby(['id','trx_tm_dayofyear'])['cny_trx_amt'].sum().reset_index(name='bmean')
temp = temp.groupby('id')['bmean'].agg(['max', 'min', 'std', 'mean']).reset_index()
temp.columns = ['id'] + ['trd_b_day_amt_' + i for i in ['max', 'min', 'std', 'mean']]
tag = tag.merge(temp, on = 'id', how = 'left')

temp = trd_c.groupby(['id','trx_tm_dayofyear'])['cny_trx_amt'].sum().reset_index(name='bmean')
temp = temp.groupby('id')['bmean'].agg(['max', 'min', 'std', 'mean']).reset_index()
temp.columns = ['id'] + ['trd_c_day_amt_' + i for i in ['max', 'min', 'std', 'mean']]
tag = tag.merge(temp, on = 'id', how = 'left')

In [31]:
for i in ['mrg_situ_cd', 'edu_deg_cd', 'acdm_deg_cd', 'deg_cd']:
    for sta in ['sum']:
        tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_{sta}'] = tag.groupby(i)['cny_trx_amt_sum_B_Dat_Flg1_Cd'].transform(sta)
        tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_{sta}'] = tag.groupby(i)['cny_trx_amt_sum_C_Dat_Flg1_Cd'].transform(sta)
#     tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_ptp'] = tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_max'] - tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_min']
#     tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_ptp'] = tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_max'] - tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_min']
#     del tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_max'],tag[f'{i}_cny_trx_amt_sum_B_Dat_Flg1_Cd_min']
#     del tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_max'],tag[f'{i}_cny_trx_amt_sum_C_Dat_Flg1_Cd_min']
# tag.drop(columns=['mrg_situ_cd', 'edu_deg_cd', 'acdm_deg_cd', 'deg_cd'], inplace=True)
tag = pd.get_dummies(columns=tag_cate_cols,data=tag)

#### **`beh`**

In [34]:
aggs = {
    'page_no':['nunique', 'count'],
    # 'trx_tm_是否周末': ['mean'],
#     'page_tm_hour': ['nunique'],
#     # 'page_tm_weekday': ['nunique'],
#     # 'trx_tm_day':['nunique'],
#     'page_tm_wy':['nunique'],
#     'page_tm_dayofyear':['nunique', 'max', 'min', np.ptp],
}
agg_df=beh.groupby('id').agg(aggs)
agg_df.columns = ['beh'+'_'+'_'.join(col).strip() for col in agg_df.columns.values]
agg_df=agg_df.reset_index()  
tag = tag.merge(agg_df, on = 'id', how='left') 

In [35]:
for i in ['beh_page_no_nunique', 'beh_page_no_count']:
    tag[i] = tag[i].fillna(0)

In [36]:
"""tfidf"""
feat = beh[['id','page_no']]
feat['page_no'] = feat['page_no'].astype(str)
feat = feat.groupby(['id'])['page_no'].agg(lambda x: ' '.join(x)).reset_index(name='page_no_list')
tf_idf = TfidfVectorizer(max_features=8)
tf_vec = tf_idf.fit_transform(feat['page_no_list'].values.tolist())
# svd = TruncatedSVD(n_components=8, n_iter=10, random_state=2019)
# svd_vec = svd.fit_transform(tf_vec)
tf_df = pd.DataFrame(tf_vec.toarray())
tf_df['id'] = feat['id'].values
tf_df.columns = ['page_no_list_svd_vec_'+str(i+1) for i in range(8)]+ ['id']
tag = tag.merge(tf_df,on='id', how='left')
del tf_df,feat;gc.collect()

2

In [37]:
for i in ['page_no_list_svd_vec_1',
       'page_no_list_svd_vec_2', 'page_no_list_svd_vec_3',
       'page_no_list_svd_vec_4', 'page_no_list_svd_vec_5',
       'page_no_list_svd_vec_6', 'page_no_list_svd_vec_7',
       'page_no_list_svd_vec_8']:
    tag[i] = tag[i].fillna(0)
#     del tag[i]

In [38]:
tag.loc[tag['dnl_mbl_bnk_ind'].isin(beh.id.unique()), 'dnl_mbl_bnk_ind'] = 1
tag['dnl_mbl_bnk_ind'].fillna(0,inplace=True)

In [39]:
train = tag[~tag['flag'].isnull()].reset_index(drop=True)
test = tag[tag['flag'].isnull()].reset_index(drop=True)

In [40]:
del_col=['beh_id','deg_cd_cny_trx_amt_sum_C_Dat_Flg1_Cd_sum',
 'cny_trx_amt_mean_1_b',
 'cny_trx_amt_median_1_b',
 'cny_trx_amt_mean_C_c',
 'cny_trx_amt_median_C_c',
 'cny_trx_amt_mean_2_c',
 'cny_trx_amt_median_2_c',
]
drop_0 = []
col=[i for i in train.columns if i not in ['id', 'flag','sample']+del_col+drop_0]
# col = tag_num_cols+cate_to_num_col
X_train=train[col].copy()#.fillna(0)
y_train=train['flag'].copy().astype(int)
X_test=test[col].copy()#.fillna(0)
print(X_train.shape,X_test.shape)

(39923, 278) (4000, 278)


### lgb

In [3]:
K =5
seed = 1111
skf = StratifiedKFold(n_splits=K, shuffle=True, random_state=seed)

In [2]:
lgb_params = {
                        'boosting_type': 'gbdt',
                        'objective': 'binary',
                        'metric': 'auc',
                        'num_leaves': 31,
                        'subsample': 0.86,
                        'max_depth':-1,
                        'colsample_bytree': 0.6,
                        'learning_rate': 0.03,
                        'bagging_freq':3,
                        'lambda_l2':2,
                        'seed': 1234,
                        'nthread': 8,
                        # 'is_unblance':True,
                        # 'gpu_device_id':0,
                        # 'gpu_platform_id':0,
                        # 'device':'gpu',
             }
lgb_models=[]
oof = np.zeros(len(X_train))
predictions = np.zeros(len(X_test))
auc_score = []

for i, (train_index, val_index) in enumerate(skf.split(X_train,y_train)):
    print("fold {}".format(i))
    X_tr, X_val = X_train.iloc[train_index].values, X_train.iloc[val_index].values
    y_tr, y_val = y_train.iloc[train_index].values, y_train.iloc[val_index].values
    
    lgb_train = lgb.Dataset(X_tr,y_tr)
    lgb_val = lgb.Dataset(X_val,y_val)
    num_round = 3000
    clf = lgb.train(lgb_params, lgb_train, num_round, valid_sets = [lgb_train, lgb_val],
                    verbose_eval=100, early_stopping_rounds = 100,
                #    categorical_feature=cate_feat
                   )#50
    lgb_models.append(clf)
    oof[val_index] = clf.predict(X_val, num_iteration=clf.best_iteration)
    pred = clf.predict(X_val, num_iteration=clf.best_iteration)
    auc_ss = roc_auc_score(y_val, pred)
    auc_score.append(auc_ss)
    print('best iteration = ',clf.best_iteration)
    print('auc = ', auc_ss)
    predictions += clf.predict(X_test.values, num_iteration=clf.best_iteration) / skf.n_splits
print('auc score : ', np.mean(auc_score), np.std(auc_score))

In [56]:
sub = test[['id']].copy()
sub['违约预测概率'] = predictions
sub.to_csv("lgb_76087.txt", sep="\t", header=None, index=None)